In [8]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Example usage:
start_description = '3rd Sunday of the month'
start_time = '6PM, Sunday'
duration_hours = 4

def get_date_range(start_description, start_time, duration_hours):
    print("Received Input:")
    print(f"Start Description: {start_description}")
    print(f"Start Time: {start_time}")
    print(f"Duration (hours): {duration_hours}")

    description_parts = start_description.split()
    
    time_parts = start_time.split(', ')
    specified_time = time_parts[0].strip()
    specified_day = time_parts[1].strip()

    start_time_24h = datetime.strptime(specified_time, "%I%p").strftime("%H:%M")

    current_date = datetime.now()

    # Finding the next Monday after the first Tuesday (Patch Tuesday)
    first_day_of_month = current_date.replace(day=1)
    first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)
    patch_tuesday = first_tuesday + timedelta(days=6)
    next_monday = patch_tuesday + timedelta(days=(7 - patch_tuesday.weekday()) % 7)
        
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_indices = {day: index for index, day in enumerate(day_names)}
    
    start_datetime = None
    end_datetime = None

    if 'next'.lower() in ' '.join(description_parts).lower():
        
        print("Executing 'Next Month' logic...")
        # Extracting the month
        current_date = current_date.replace(day=1) + relativedelta(months=1)
        
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(start_description.split()[0][:-len("th")])
        
        # Calculating the selected weekday of the next month
        first_day_of_next_month = current_date.replace(day=1)
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_day_of_next_month.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_day_of_next_month + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        print("\nFinal Calculated Dates:")
        print(f"Start Date: {start_datetime.strftime('%Y-%m-%d %H:%M')}")
        print(f"End Date: {end_datetime.strftime('%Y-%m-%d %H:%M')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M"), end_datetime.strftime("%Y-%m-%d %H:%M")
        
    elif 'after'.lower() in ' '.join(description_parts).lower():

        print("Executing 'after Patch Tuesday' logic...")
        if description_parts[0] == '0':
            start_day_index = day_indices[specified_day.capitalize()] - next_monday.weekday()
        else:
            # Extracting the week offset
            week_offset = int(description_parts[0][:-2])
            # Calculate the starting day index within the desired week after Patch Tuesday
            start_day_index = (day_indices[specified_day.capitalize()] - next_monday.weekday() + 7) % 7
            # Adjust to the specified week within the month
            start_day_index += week_offset * 7
            # Adjust if the start_day_index is negative
            if start_day_index < 0:
                start_day_index += 7
        
        # Constructing start date and time
        start_date = next_monday + timedelta(days=start_day_index)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        print("\nFinal Calculated Dates:")
        print(f"Start Date: {start_datetime.strftime('%Y-%m-%d %H:%M')}")
        print(f"End Date: {end_datetime.strftime('%Y-%m-%d %H:%M')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M"), end_datetime.strftime("%Y-%m-%d %H:%M")

    elif 'of the month'.lower() in ' '.join(description_parts).lower():

        print("Executing 'of the' logic...")
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(description_parts[0][:-len("th")])

        # Finding the current month's first day
        first_day_of_month = current_date.replace(day=1)

        # Finding the desired weekday of the month
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_day_of_month.weekday()

        # Calculate the starting day index within the adjusted month
        diff = (selected_weekday - current_weekday + 7) % 7
        start_date = first_day_of_month + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Check if the calculated date is beyond the end of the month
        if start_date.month != first_day_of_month.month:
            start_date -= timedelta(days=7)

        # Adjusting for leap year if the selected month is February
        if start_date.month == 2 and start_date.day > 28:
            # Check if it's a leap year
            if (start_date.year % 4 == 0 and start_date.year % 100 != 0) or (start_date.year % 400 == 0):
                # Adjust to the last day of February in a leap year
                start_date = start_date.replace(day=29)


        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        print("\nFinal Calculated Dates:")
        print(f"Start Date: {start_datetime.strftime('%Y-%m-%d %H:%M')}")
        print(f"End Date: {end_datetime.strftime('%Y-%m-%d %H:%M')}")

        return start_datetime.strftime("%Y-%m-%d %H:%M"), end_datetime.strftime("%Y-%m-%d %H:%M")

    else:
        # Additional cases of weekday logic...
        return "Error: Invalid description", ""

start_date_formatted, end_date_formatted = get_date_range(start_description, start_time, duration_hours)
if start_date_formatted and end_date_formatted:
    print("\nFinal Output:")
    print(f"Start Date: {start_date_formatted}")
    print(f"End Date: {end_date_formatted}")


Received Input:
Start Description: 3rd Sunday of the month
Start Time: 6PM, Sunday
Duration (hours): 4
Executing 'of the' logic...

Final Calculated Dates:
Start Date: 2024-02-18 18:00
End Date: 2024-02-18 22:00

Final Output:
Start Date: 2024-02-18 18:00
End Date: 2024-02-18 22:00
